In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [ ]:
import os
# TODO: Change this to your Drive folder location
data_dir = '/content/drive/My Drive/github/object_detection/test/'
os.chdir(data_dir)
!pwd

# import some common libraries
import numpy as np
import cv2, random
from PIL import Image
from google.colab.patches import cv2_imshow

# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from torchvision import transforms

In [ ]:
# select model (get_detector)
cfg = get_cfg()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# "/COCO-InstanceSegmentationn/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
# "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# "/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
# "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)

In [ ]:
# load images: adversarial images are background images not in the COCO dataset and used as a patch
src_r = Image.open('000000001192.jpg')
adv_r = Image.open('adversarial2.jpg')  # sample training data from COCO

# check our loaded images
src.show()
adv.show()

In [ ]:
preprocess = transforms.Compose([transforms.Resize(800), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
# resize the adversarial image(src) to match the dimensions of your target image
# beware that cv2.resize uses different convention from images in numpy arrays # numpy : (height, width, 3) # image : width x height

src = preprocess(src_r)
adv = preprocess(adv_r)
src = torch.unsqueeze(img, 0)
adv = torch.unsqueeze(adv, 0)

In [ ]:
# raw predictions
outputs = predictor(src)

print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(src[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

# Experiments

In [ ]:
# experiment 1: replace top of the image with differnt background
# set grid_num, which is the number of grids that you want to equally divide the height, width of the image
grid_num = 12
grid_h = height//grid_num
print(grid_h)

img_list = []

for i in range(5):
  src_cp = np.copy(src)
  src_cp[:grid_h*i,:] = adv[:grid_h*i,:]
  cv2_imshow(src_cp)
  img_list.append(src_cp)

In [ ]:
# experiment 2: replace bottom of the image with differnt background
# set grid_num, which is the number of grids that you want to equally divide the height, width of the image
grid_num = 12
grid_h = height//grid_num
print(grid_h)

img_list = []

for i in range(5):
  src_cp = np.copy(src)
  src_cp[height-grid_h*i:,:] = adv[height-grid_h*i:,:]
  cv2_imshow(src_cp)
  img_list.append(src_cp)

In [ ]:
# experiment 3: 
# set grid_num, which is the number of grids that you want to equally divide the height, width of the image
grid_num = 24
grid_h = height//grid_num
print(grid_h)

img_list = []

for i in range(10):
  des_cp = np.copy(adv)
  des_cp[ymin:int(ymin+2*mod_unit*i),xmin:int(xmin+4*mod_unit*i)] = src[ymin:int(ymin+2*mod_unit*i),xmin:int(xmin+4*mod_unit*i)]
  cv2_imshow(des_cp)
  img_list.append(des_cp)

In [ ]:
# experiment 3: gradually hiding portions of a dog
# set grid_num, which is the number of grids that you want to equally divide the height, width of the image
grid_num = 24
grid = height//grid_num
print(grid)

img_list = []

start_h = 80
start_w = 100

for i in range(10):
  src_cp = np.copy(src)
  src_cp[start_h:int(start_h+2*i*grid), start_w:int(start_w+2*i*grid)] = adv[start_h:int(start_h+2*i*grid), start_w:int(start_w+2*i*grid)]
  cv2_imshow(src_cp)
  img_list.append(src_cp)

In [ ]:
# experiment 4,5: putting the dog in a different background
# set grid_num, which is the number of grids that you want to equally divide the height, width of the image
grid_num = 24
grid = height//grid_num
print(grid)

img_list = []

start_h = 80
start_w = 100

for i in range(10):
  des_cp = np.copy(adv)
  des_cp[start_h:int(start_h+2*i*grid), start_w:int(start_w+2*i*grid)] = src[start_h:int(start_h+2*i*grid), start_w:int(start_w+2*i*grid)]
  cv2_imshow(des_cp)
  img_list.append(des_cp)

In [ ]:
# experiment 6-1: gradual random perturbation of the pixels
img_list = []
mu = 0
sigma = 1

src_cp = np.copy(src)
img_list.append(src)

for i in range(1000001):
  rand_h = np.random.randint(low=0, high=src.shape[0])
  rand_w = np.random.randint(low=0, high=src.shape[1])
  rand_d = np.random.randint(low=0, high=src.shape[2])
  
  src_cp[rand_h, rand_w, rand_d] = src_cp[rand_h, rand_w, rand_d] + np.random.normal(mu, sigma)
  
  if i % 200000 == 0:
    print(i)
    img_list.append(src_cp)
    cv2_imshow(src_cp)

print('FINISHED')

In [ ]:
# experiment 6-2: grid random perturbation of the pixels
grid_num = 10
grid = height//grid_num
print(grid)

img_list = []
mu = 0
sigma = 1

src_cp = np.copy(src)
for _ in range(10):
  rand_h = np.random.randint(low=0, high=src.shape[0])
  rand_w = np.random.randint(low=0, high=src.shape[1])
  src_cp[rand_h, rand_w] = src_cp[rand_h, rand_w] + np.random.normal(mu, sigma)
  
  img_list.append(src_cp)
  cv2_imshow(src_cp)

print('FINISHED')

In [ ]:
# experiment 6-3: entire random perturbation of the pixels
img_list = []
mu = 10
sigma = 5

img_list.append(src)
src_cp = np.copy(src)
src_cp = src_cp + np.random.normal(mu, sigma, (src.shape[0], src.shape[1], src.shape[2]))  
img_list.append(src_cp)

print('FINISHED')

In [ ]:
# experiment 7: hiding only the eye and nose of a dog
img_list = [] # (y,x) = (height, width)

left_eye = (180, 200, 215, 240) 
right_eye = (180, 200, 255, 280) 
nose = (230, 250, 235, 260) 

src_cp = np.copy(src)
src_cp[left_eye[0]:left_eye[1],left_eye[2]:left_eye[3]] = 0
src_cp[right_eye[0]:right_eye[1],right_eye[2]:right_eye[3]] = 0
src_cp[nose[0]:nose[1],nose[2]:nose[3]] = 0

cv2_imshow(src_cp)
img_list.append(src_cp)

# Run inference on adversarial examples

In [ ]:
for img in img_list:
    outputs = predictor(img)
    
    # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
    print(outputs["instances"].pred_classes)
    print(outputs["instances"].pred_boxes)

    # We can use `Visualizer` to draw the predictions on the image.
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2_imshow(out.get_image()[:, :, ::-1])